In [1]:
import requests
from copy import copy


def dfs(category_id):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:

        d_id = d['id']
        d_name = d['name']
        d_is_leaf = d['is_leaf']
        
        if(d_is_leaf):
            leaf_id = copy(d_id)
            leaf_name = copy(d_name)
            leaf_category.append([leaf_id,leaf_name])
            
        else:
            dfs(d_id)
    
paths = []
leaf_category = []
starting_category = [1,2,3,4,5,8,9] #출산/육아(6) , 식품 제외(7)


dfs(4)

print(len(leaf_category))
#1 : 패션의류 --> 68개 
#2 : 패션잡화 --> 264개
#3 : 화장품/미용 --> 157개
#4 : 디지털/가전 --> 681개
#5 : 가구/인테리어 --> 354개
#6 : 출산/육아 (X)
#7 : 식품 (X)
#8 : 스포츠/레저 --> 카테고리 : 480개, 키워드 : 154207개 --> 필터 적용 : 20000개
#9 : 생활/건강 --> 카테고리 : 1227개, 키워드 : 465817개 --> 필터 적용 : 61751개

#app 1개당 기본 api 12500개 
#계정 1개당 app 10개, api 125000개

#계정 2개면 충분!

681


In [2]:
import json
import csv
import urllib.request
from tqdm import notebook
from joblib import Parallel, delayed
from config import *
from infsell import *



def extract_keywords(category,client_auth):
    
    category_id = category[0]
    category_name = category[1]
    
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(category_id)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(category_id)
    response_data = requests.post(url_data, json=PAYLOAD)
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data']
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords,dict_clicks = get_keywords(data)
    keywords_nb = remove_brands(keywords,brands)
    
    for keyword in notebook.tqdm(keywords_nb, desc = "키워드 진행도"):

        encText = urllib.parse.quote(keyword)
        url_excluding_oversea = "https://openapi.naver.com/v1/search/shop.json?display=80&exclude=cbshop&query=" + encText
        url_items = "https://openapi.naver.com/v1/search/shop.json?display=80&query=" + encText # JSON 결과

        items_excluding_oversea = get_items(url_excluding_oversea,client_auth)
        items = get_items(url_items,client_auth)

        dict_excluding_oversea = {}

        for item in items_excluding_oversea:
            dict_excluding_oversea[item['productId']]=1
        
        cnt = 0 
        for item in items :
            if item['productId'] not in dict_excluding_oversea:
                cnt+=1
    
        info_keyword={}
        info_keyword['카테고리'] = category_name
        info_keyword['키워드'] = keyword
        info_keyword['검색수'] = dict_clicks[keyword]
        info_keyword['해외배송개수'] = cnt
        
        final_keywords.append(info_keyword)
        

#멀티코어 개수
num_core = 12

cnt_keyword=0
path_auth='auth_api.txt'
client_auth = get_auth_api(path_auth)
final_keywords = []

with Parallel(n_jobs=num_core, require='sharedmem') as parallel:
    parallel(delayed(extract_keywords)(category,client_auth)for category in notebook.tqdm(leaf_category))
    
print(final_keywords)

  0%|          | 0/681 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/10 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/4 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/16 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/19 [00:00<?, ?it/s]

...

키워드 진행도:   0%|          | 0/48 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/15 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/229 [00:00<?, ?it/s]

키워드 진행도:   0%|          | 0/185 [00:00<?, ?it/s]

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [3]:
from infsell import convert_dict_to_excel
import openpyxl

name_file = '2023_06_08_top500_키워드'

convert_dict_to_excel(name_file, final_keywords)

엑셀 변환 완료
